In [89]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [3]:
URL = 'https://wiki.radioreference.com/index.php/Davidson_County_(TN)'

In [5]:
response = requests.get(URL)

In [7]:
soup = BS(response.text)

In [59]:
# html looks like:
# <h3><span class="mw-headline" id="Ten_Codes">Ten Codes</span></h3>
# <pre>-10-1 Receiving Poorly
# -2 Receiving Well
# -3 special assignment

# Looking for the tag <h3> that contains <span class="mw-headline" id="Ten_Codes">, followed by a <pre>.
# I want the contents of the <pre>

In [73]:
possibles = []
i = 0

for h in soup.findAll('h3') :
    if 'Ten_Codes' in str(h) :
        possibles.append(i)
        #print(h.find_next_sibling('pre'))
    i += 1

if len(possibles) == 0 :
    print('No results.  Look into the html again...')
elif len(possibles) > 1 :
    print('More than 1 result.  ' + str(len(possibles)) + ' found.  Look into the html again...')
else :
    print('Found it!  Assigned to p.')
    p = soup.findAll('h3')[possibles[0]].find_next_sibling('pre')

Found it!  Assigned to p.


In [ ]:
# get the first number before the first space, and everything after the first space (in proper case)
# the first several examples are:
# '-10-1 Receiving Poorly',      #  -- this one is unique, since it shows the 10 in 10-1.  Disregard that, for consistency.
# '-2 Receiving Well',
# '-3 special assignment'

In [101]:
df = pd.DataFrame(columns = ['tencode','description'])

In [103]:
for line in p.text.splitlines() :
    match = re.search(r'(\d+)\s(.+)', line)    # search for a digit or multiple, followed by a space, followed by any number of any characters
    if match :
        code = match.group(1)
        description = match.group(2).title()  # proper case it
        new_row = {'tencode': code,
                   'description': description}
        df.loc[len(df)] = new_row
    else :
        print('Missed a match in a row... look into this and try again?')
        print('Error when trying to match the line:')
        print(line)

In [107]:
# look at the df
df

,tencode,description
0,1,Receiving Poorly
1,2,Receiving Well
2,3,Special Assignment
3,4,Acknowledgement
4,7,Out Of Service
...,...,...
62,95,Lunch
63,96,Business Check
64,97,Arrived On Scene
65,98,Assignment Complete


In [109]:
# save the csv
df.to_csv('../data/tencodes.csv', index=False)